# WEEK 6 : Data Preparation

## WEEK 5 : HTML: 웹 내용 긁어오기(Web Crawling)

In [ ]:
#-*- coding: euc-kr-*-

# 웹페이지에 보이는 내용 역시 프로그래밍 된 결과라고 이해할 수 있습니다.
# 웹 크롤링이란, HTML 형식으로 작성되어 있는 웹 페이지 내용을 가져와 웹 페이지의 내용을 파악하는 작업을 뜻합니다.

# 실습으로 네이버 웹툰 페이지의 내용을 읽어와서 웹툰 목록을 만들어 보겠습니다.

import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

ret = requests.get('http://comic.naver.com/webtoon/weekday.nhn')
soup = bs(ret.text, "html.parser")
titles = soup.find_all("a", "title")

df = pd.DataFrame()
for title in titles :
    buf = {
        'title': title['title'].encode('utf-8'),
        'link' : title['href']
    }
    s = pd.Series(buf)
    df = df.append(s, ignore_index=True)

print df

## 데이터 불러오기

### (1) 데이터 로딩 - 지진 데이터

In [ ]:
#-*- coding: utf-8 -*-
import pandas as pd

earthquake = pd.read_table('./data/earthquake_data.csv', sep=',', encoding = 'euc-kr')
print earthquake.head()
print earthquake.tail()

### (2) Handling Missing Data

In [ ]:
# 비어있는 데이터가 있는지 체크해보기
import numpy as np
import pandas as pd

stringData = pd.Series(['apple', 'melon', 'grape', None]) # 혹은 numpy.nan
print stringData

print stringData.isnull()

stringData[0] = None
print stringData.isnull()
print stringData.notnull()

# 비어있는 데이터를 어떻게 처리할까요?

# 1) 빈 데이터는 불러오지 않는다.
print stringData.dropna()
print stringData

# 1차원 Series는 간단했지만 2차원 DataFrame을 생각하면 조금 복잡해집니다.
df = pd.DataFrame([[1, 2.3, 4], [2, None, 3.8], [None, None, None], [None, 7.9, 4]])
print df.isnull()

print df.dropna() # 행을 기준으로 None 값이 하나라도 존재하면 drop 합니다.
print df.dropna(how='all') # 행의 모든 값이 None이면 drop 합니다.

# 열을 기준으로 하고 싶을때는?
print df.dropna(axis = 1) # default 0은 행 기준, 1은 열 기준
print df.dropna(axis = 1, how = 'all')

# 2) 빈 데이터를 다른 값으로 대체한다.
# 비어있는 값 때문에 한 행을 통째로 버리는 게 때로는 비합리적일 수 있습니다.
# 비어있는 값을 적절한 다른 값으로 대체하는 게 데이터의 왜곡을 줄여줄 수도 있죠.

print df.fillna(0)
# 열 별로 다른 값을 넣을 수도 있습니다.
print df.fillna({1: 0.5, 2: 3})

# None 값 위 또는 아래에 있는 값으로 대체할 수도 있습니다.
print df.fillna(method = 'ffill') # forward fill
print df.fillna(method = 'bfill') # backward fill

print df.fillna(df.mean()) # 이런 값도 가능하겠죠?

In [ ]:
# 그럼 우리가 불러온 데이터를 직접 정리해볼까요
# 그 전에 데이터에 빈 값이 있는지 구해봅시다.
import numpy as np

len(earthquake) # 총 데이터 행 수
print earthquake.isnull() # 데이터가 너무 많아서 보이지가 않죠

# 실제로 빈칸이 있는지 없는지 눈으로 확인해봅시다.

print earthquake.isnull().sum() # True = 1, False = 0인 성질을 이용하였습니다.

earthquake = earthquake.replace('-', np.nan)

# 위치 정보가 약 23개 정도 비어 있네요.
# 여기서 여러분이 판단하셔야 하는거죠. 
# 1223개의 데이터 중에 23 정도는 무시해도 될지
# 아니면 진원시나 규모 등 다른 데이터는 의미 있는 값이니 비어있는 값을 처리하여 그대로 사용할지.

# 두가지를 모두 해보겠습니다.

# 1) nan 값이 있는 행 버리기
earthquake_wo_nan = earthquake.dropna()
print len(earthquake_wo_nan) # 1223개 데이터 중에서 23개 만큼 버렸습니다.
earthquake_wo_nan.isnull().sum() # 모두 값이 차 있는 걸 볼 수 있죠

# 2) 특정 값으로 대체하기
# csv/txt 파일을 불러올 때 비어있는 값은 자동으로 NaN 값으로 들어가는 것을 확인했습니다.
# 비어있는 값을 그대로 NaN로 처리하고 싶다면 바로 사용하면 되지만,
# 때에 따라서는 다른 값으로 대체하고 싶은 경우가 있겠죠?
# 보통 유효하지 않은 값은 0이나 -1 같은 값으로 자주 사용하는데요.
# 이런 경우에는 데이터에 왜곡이 생기지 않도록 값을 잘 정하는 것이 중요합니다.

# 예를 들어 볼까요?
# 위의 예에서 NaN 값이 있는 column은 위도와 경도입니다.
# 원래 위도와 경도의 범위는 각각 +/-90, +/-180 이죠.
# 그런데 우리가 가지고 있는 데이터는 한국의 지진데이터이기 때문에 데이터의 범위가 좀 더 좁혀질 것 같습니다.

print earthquake[u'위도'].dropna().max()
print earthquake[u'위도'].dropna().min()
print earthquake[u'경도'].dropna().max()
print earthquake[u'경도'].dropna().min()

# 위도는 32~41도, 경도는 122~131도에 분포되어 있네요.
# 이런 경우에는 0 혹은 -1 값으로 대체해도 합리적이겠네요
earthquake.fillna(0)
earthquake.fillna(-1)

# 그런데 만약 데이터가 전세계 지진 데이터였다면 0으로 대체하는 것은 부적절할 수 있겠죠?
# 실제 (0,0)에서 지진이 나는 경우가 있을 수 있으니까요.

# 값을 대체할 때는 데이터의 왜곡이 생기지 않도록 신경써주셔야 합니다.


## 2. 데이터 준비하기

우리가 위의 데이터를 가지고 최종적으로 할 일은 데이터를 정리해서 다음주에 다양한 방법으로 시각화 하는 것입니다.

한국의 지진 발생 추이를 그래프로 그려볼 수도 있고, 남북한의 지진 발생 정도를 비교할 수도 있을 것입니다.
지도에 지진 발생 지점을 표시해 한 눈에 지리적 차이에 따른 지진 발생의 차이를 알아볼 수도 있을 것입니다.

이렇게 자료를 가지고 우리가 원하는 결과를 얻기 위해서는 가지고 있는 데이터를 처리하기 편하게 정리하거나
혹은 데이터를 활용해서 제 3의 데이터를 만들어내는 과정이 필요합니다.

이번 시간에는 위의 지진 데이터를 활용해서 우리가 원하는 방식으로 데이터를 다듬는 연습을 해보겠습니다.

### (1) 진원시

In [ ]:
import numpy as np
earthquake = earthquake.replace('-', np.nan)

print earthquake[u'진원시'].head()

# 현재 진원시 열에 있는 데이터 타입을 확인해보겠습니다.
type(earthquake[u'진원시'][0])

# 지금은 단순한 문자열 타입이라서 연도 / 월 / 일 등을 구분하려고 하면
# 아래처럼 문자열을 split으로 나누어야 합니다.
# 복잡하기도 하고 가독성도 떨어지죠?
date_str = earthquake[u'진원시'][0].replace(':', '-').replace(' ', '-').split('-')
print date_str[0] # Year
print date_str[1] # Month
print date_str[2] # Day

# 이 문자열을 시간 표시형식으로 변환시키면 훨씬 다루기가 쉬워질 것입니다.

from datetime import datetime 

date = datetime.strptime(earthquake[u'진원시'][0], '%Y-%m-%d %H:%M')
print date.year
print date.month
print date.day

# 언뜻보면 출력값은 같지만 타입을 비교해보면
type(date_str[0])
type(date.year)

# 문자열과 정수 타입으로 다르죠? 연도 / 월 / 일 / 시간 비교에 정수 타입이 훨씬 직관적인 것은 당연합니다.
# 그럼 우리 데이터에 있는 날짜데이터를 다 해당 형식으로 바꿔봅시다.

# Column에 공통적으로 특정 작업을 수행하려면?

# apply 함수 : Column에 공통적으로 특정 함수를 적용할 때 사용하는 방법입니다.
def string_to_datetime(string) :
    return datetime.strptime(string, '%Y-%m-%d %H:%M')

earthquake[u'진원시_apply'] = earthquake[u'진원시'].apply(string_to_datetime)
print earthquake.head()
print type(earthquake[u'진원시_apply'])


# lambda 축약함수 : 간단한 형태의 함수를 한 줄로 나타내주는 기능을 합니다
# 형태 : lambda 인자 : 표현식
def add (a, b):
    return a+b

# a와 b의 합을 리턴하는 간단한 형태의 함수가 있다고 가정해보겠습니다.
print add(10, 30) # 일반적인 함수의 형태는 이렇게 호출했었죠?

# 람다 형식으로 바꾸어 사용해보겠습니다.

print (lambda a, b : a+b)(10, 30) # (람다 표현식)(인자)

# 그럼 위에서 사용한 string_to_datetime이라는 간단한 함수도 람다로 표현해볼까요?

# 진원시 열 전체에 lambda 표현식을 적용하기 위해서는 위에서와 마찬가지로 apply 함수를 사용해야 합니다.
earthquake[u'진원시_lambda'] = earthquake[u'진원시'].\
    apply(lambda string : datetime.strptime(string, '%Y-%m-%d %H:%M'))

# 전체 Series에 함수를 적용해주는 map이라는 함수도 있습니다.
earthquake[u'진원시_map'] = map(lambda string : datetime.strptime(string, '%Y-%m-%d %H:%M'), earthquake[u'진원시'])
print earthquake[u'진원시_map'].head()

# 람다의 장점
# 함수의 가장 큰 특징은 재사용성에 있죠. 간단한 함수라고 하더라도 다른 식에서 자주 사용하는 함수라면
# 람다 형태로 바꾸기 보다는 함수 형태로 정의를 하는 것이 좋을 것입니다.


# 현재 진원시 데이터는 YYYY-MM-DD HH:mm 형태로 저장되어 있습니다.
# 그런데 시간 정보는 우리가 분석하려는 내용이 크게 쓰이지 않을 것 같습니다.
# 물론 어느 시간대에 지진이 가장 많이 일어났는지 분석하고 싶다면 반드시 필요한 정보이지만
# 시간이 지진 발생에 중요한 factor는 아닐 것이라고 '가정'하고
# 일단은 필요없는 시간 정보를 지워버리는 게 편할 것 같네요.


def drop_time(date_time) :
    return datetime(date_time.year, date_time.month, date_time.day)

earthquake[u'진원시_date'] = earthquake[u'진원시_apply'].apply(drop_time)
print earthquake[u'진원시_date'].head()


# 그 동안 연습하느라 만들어진 column들을 지우고 최종 결과물만 남겨놓겠습니다.
earthquake_drop = earthquake.drop([u'진원시',u'진원시_apply', u'진원시_lambda', u'진원시_map'] , axis = 1)
print earthquake_drop.head()

# 아니면 원하는 column만 선택해서 새로운 DataFrame을 만들 수도 있겠죠?
earthquake_final = pd.DataFrame(earthquake, columns = [u'진원시_date', u'규모', u'위도', u'경도', u'위치'])
earthquake_final.columns = [u'진원시', u'규모', u'위도', u'경도', u'위치']
print earthquake_final.head()


# Duplicated data

# 일단 중복되는 데이터가 있는지 검사를 해봅시다.
earthquake_final.duplicated() # 중복되는 데이터가 있는지 검사해줍니다.
earthquake_final.duplicated().sum()

# 중복되는 데이터가 무엇인지 어떻게 찾아낼 수 있을까요?
# Data Selecting : 조건에 맞는 데이터만 골라낼 수 있습니다!

# DataFrame[조건] : 조건이 참인 행만 골라냅니다
s = pd.Series(range(0, 10))
print s

s[s > 5]

# |  &  ~
s[(s > 9) | (s < 2) ]
s[(s > 5) & (s <= 7)]
s[~(s*2 > 10)]


# earthquake_final에서 중복되는 데이터만 골라서 출력할 수 있겠죠?

print earthquake_final[earthquake_final.duplicated()] # '== True'가 생략된 형태겠죠.

# 중복된 데이터를 버려도 될까요?
earthquake_final.drop(earthquake_final[earthquake_final.duplicated()].index)
earthquake_final_final = earthquake_final.drop_duplicates()
print earthquake_final_final.duplicated().sum()
print earthquake_final_final.head()


# 지금까지의 결과물을 다시 TXT 파일로 저장해놓겠습니다.

earthquake_final.to_csv('./data/earthquke_time_final.txt', sep = ',', header = earthquake_final.columns,
                        index = None, encoding = 'utf-8')


### (2)  규모

In [ ]:
earthquake = pd.read_csv('./data/earthquke_time_final.txt', sep = ',', encoding = 'utf-8')

# 규모는 리히터 규모 기준으로, 실수로 표현되어 있습니다

# 규모를 기준으로 오름차순/내림차순으로 정렬해볼까요
# Ranking and Sorting
print earthquake.sort_values(by = u'규모').head()

# 내림차순으로 정렬하고 싶다면
print earthquake.sort_values(by = u'규모', ascending = False).head()

# 기준을 여러개 설정할 수 있습니다.
# 규모를 기준으로 1차 내림차순 정렬을 하고, 규모가 동일할 경우에는 진원시를 기준으로 오름차순 정렬해보겠습니다.
print earthquake.sort_values(by = [u'규모', u'진원시'], ascending = [False, True]).head()
print type(earthquake[u'규모'][0])


# 아래의 구분을 기준으로 한국에서 발생한 지진을 micro ~ massive로 구분해봅시다.
# Apply

"""
(참고) 리히터 규모의 힘 단위(출처: 위키피디아)

2.0 이하 - micro : 지진계가 감지할 수 있는 정도
2.1 ~ 3.9 - minor : 가끔 느끼지만 거의 영향은 없음, 땅이 조금 흔들리는 정도 (여진)
4.0 ~ 4.9 - light : 실내 물건들의 느낄 수 있는 수준의 덜컹거림, 땅이 조금 흔들리는 정도 (여진)
5.0 ~ 5.9 - moderate : 약한 건물들은 피해를 받을 수 있음, 전봇대가 파손되는 정도
6.0 ~ 6.9 - strong : 땅이 뚜렷하게 흔들리고 주택등이 무너지는 정도
7.0 ~ 7.9 - major : 땅이 심하게 흔들리는 정도, 아파트 등 큰 빌딩이 무너지는 정도
8.0 ~ 9.9 - great : 땅이 심하게 흔들리는 정도, 아파트 등 큰 빌딩이 무너지는 정도
10.0 이상 - massive : 한번도 발생한 적 없음
"""
# '강도' 열을 추가하여 해당 열에는 위의 구분대로 micro~massive 값을 대입합니다.
def eq_level(rate) :
    limit = [2.0, 3.9, 4.9, 5.9, 6.9, 7.9, 9.9]
    level = ['micro', 'minor', 'light', 'moderate', 'strong', 'major', 'great']

    for i in range(len(limit)) :
        if rate <= limit[i]:
            return level[i]
   
    return 'massive'

def earthquake_rate(rate) :
    if rate > 10 :
        return 'massive'
    elif rate >= 8.0 :
        return 'great'
    elif rate >= 7.0:
        return 'major'
    elif rate >= 6.0:
        return 'strong'
    elif rate >= 5.0 :
        return 'moderate'
    elif rate >= 4.0 :
        return 'light'
    elif rate >= 2.1 :
        return 'minor'
    else:
        return 'micro'

earthquake[u'강도'] = earthquake[u'규모'].apply(earthquake_rate)


# 강도에 따라 그룹핑
# Group by
earthquake_group = earthquake.groupby(u'강도')

# Iterating through groups
for name, group in earthquake_group :
    print name
    print group
    
    
# Selecting a group
earthquake_group.get_group('moderate')

# group별 데이터 개수
print earthquake_group.size()


### (3) 위도 / 경도

In [ ]:
# 위도 경도 데이터를 보시면 뒤에 N / E가 붙어있죠?
# 위도 값의 범위 : +90.00000(North)북위 90도 ~ -90.000000(South)남위 90도 
# 경도 값의 범위 : +180.000000(East)동경 180도 ~ -180.000000(West)서경 180도 [그리니치 천문대 기준 0도]
print type(earthquake[u'위도'][0])
print earthquake[u'위도'][0]
# N 은 양수 S는 음수
# E는 양수 W는 음수로 바꿔봅시다
def convert_latlon(string):
    data = string.split(' ')
    if data[0] == u'0':
        return 0
    elif (data[1] == u'N') or (data[1] == u'E'):
        return float(data[0])
    else :
        return -(float(data[0]))


def convert_latitude(string):
    
    if string == '-' :
        return 0
    
    lat = string.split(' ')
    
    if lat[1] == u'S':
        return -(float(lat[0]))
    elif lat[1] == u'N':
        return float(lat[0])

    
def convert_long(string) :
    
#     if type(string) == float :
#         return 0
    longitude = string.split(' ')
    if longitude[1] == u'W':
        return -(float(longitude[0]))
    elif longitude[1] == u'E':
        return float(longitude[0])  
    
earthquake[u'위도'] = earthquake[u'위도'].apply(convert_latitude)
earthquake[u'경도'] = earthquake[u'경도'].apply(convert_long)

print earthquake.head()
# 비어 있는 데이터는 여러분 재량껏 다뤄주세요



### (4) 위치

In [ ]:
# 진원지의 위치를 나타내는 column입니다.
# 자세히 보시면 남한의 데이터 뿐 아니라 북한의 데이터도 섞여있는 것을 볼 수 있는데요
# 주소를 기준으로 남/북한을 구분하여 새로운 열에 정리해봅시다.

def south_or_north(string):
    loc = string.split(' ')
    return loc[0]

earthquake[u'지역'] = earthquake[u'위치'].apply(south_or_north)

print earthquake.head()

earthquake.to_csv('./earthquake_preparation.txt', sep = ',', encoding='utf-8', index=None)